In [1]:
!pip install psycopg2-binary pandas sqlalchemy

In [2]:
import pandas as pd

employment_non_regular_by_region = pd.read_csv('datasets/Total Employment of Non-Regular Workers by Region.csv')
employment_non_regular_by_industry = pd.read_csv('datasets/Total Employment of Non-Regular Workers by Industry Group.csv')
employment_non_regular_by_business_size = pd.read_csv('datasets/Total Employment of Non-Regular Workers by Business Size.csv')


In [3]:
# Drop rows with all NaN values
def drop_null_values(df):
    df_cleaned = df.dropna(how='all').reset_index(drop=True)
    return df_cleaned

employment_non_regular_by_business_size = drop_null_values(employment_non_regular_by_business_size)
employment_non_regular_by_industry = drop_null_values(employment_non_regular_by_industry)
employment_non_regular_by_region = drop_null_values(employment_non_regular_by_region)


In [4]:
# Drop duplicates
employment_non_regular_by_region.drop_duplicates(inplace=True)
employment_non_regular_by_industry.drop_duplicates(inplace=True)
employment_non_regular_by_business_size.drop_duplicates(inplace=True)

employment_non_regular_by_region.fillna(0, inplace=True)
employment_non_regular_by_industry.fillna(0, inplace=True)
employment_non_regular_by_business_size.fillna(0, inplace=True)


In [5]:
# Add gender_category in Region dataset if its missing
if 'gender_category' not in employment_non_regular_by_region.columns:
    region_name_index = employment_non_regular_by_region.columns.get_loc('Region') + 1
    employment_non_regular_by_region.insert(region_name_index, 'Gender Category', None)
    

In [6]:
import pandas as pd

columns_to_fix = [
    'Year',
    'Total Non-Regular Workers', 
    'Probationary Workers', 
    'Casual Workers', 
    'Contractual/ Project-based Workers', 
    'Seasonal Workers', 
    'Apprentices/ Learners'
]

def remove_decimals(df):
    for column in columns_to_fix:
        if column in df.columns:
            df[column] = df[column].astype(int)
    return df

employment_non_regular_by_region = remove_decimals(employment_non_regular_by_region)
employment_non_regular_by_industry = remove_decimals(employment_non_regular_by_industry)
employment_non_regular_by_business_size = remove_decimals(employment_non_regular_by_business_size)



In [7]:
# Different categories and diff employment
other_category = {
    'Gender Category': 'gender_category',
    'Year': 'year',
    'Total Non-Regular Workers': 'total_non_regular',
    'Probationary Workers': 'probationary',
    'Casual Workers': 'casual',
    'Contractual/ Project-based Workers': 'contractual',
    'Seasonal Workers': 'seasonal',
    'Apprentices/ Learners': 'apprentices_learners'
}

# Rename columns for Regions dataset
employment_non_regular_by_region.rename(columns={
    'Region': 'region_name', 
    **other_category
}, inplace=True)

# Rename regions
region_mapping = {
    'Philippines': 'ph',
    'National Capital Region': 'ncr',
    'Cordillera Administrative Region': 'car',
    'Region I - Ilocos Region': 'region_i',
    'Region II - Cagayan Valley': 'region_ii',
    'Region III - Central Luzon': 'region_iii',
    'Region IV-A - CALABARZON': 'region_iv_a',
    'MIMAROPA Region': 'region_iv_b',
    'Region V - Bicol Region': 'region_v',
    'Region VI - Western Visayas': 'region_vi',
    'Region VII - Central Visayas': 'region_vii',
    'Region VIII - Eastern Visayas': 'region_viii',
    'Region IX - Zamboanga Peninsula': 'region_ix',
    'Region X - Northern Mindanao': 'region_x',
    'Region XI - Davao Region': 'region_xi',
    'Region XII - SOCCSKSARGEN': 'region_xii',
    'Region XIII - CARAGA': 'region_xiii',
    'Bangsamoro Autonomous Region in Muslim Mindanao': 'barmm_region'
}
employment_non_regular_by_region['region_name'] = employment_non_regular_by_region['region_name'].replace(region_mapping)

# Rename columns for the industry dataset
employment_non_regular_by_industry.rename(columns={
    'Industry Group': 'industry_category',
    **other_category
}, inplace=True)

# Naming for industries
industry_mapping = {
    'All Industries': 'all_industries',
    'Agriculture, Forestry and Fishing': 'agriculture_forestry_fishing',
    'Mining and Quarrying': 'mining_quarrying',
    'Manufacturing': 'manufacturing',
    'Electricity, Gas, Steam and Air Conditioning Supply': 'electricity_gas_steam_air_conditioning',
    'Water Supply, Sewerage, Waste Management and Remediation Activities': 'water_supply_sewerage_waste_management',
    'Construction': 'construction',
    'Wholesale and Retail Trade, Repair of Motor Vehicles and Motorcycles': 'wholesale_retail_trade',
    'Transportation and Storage': 'transportation_storage',
    'Accommodation and Food Service Activities': 'accommodation_food_service',
    'Information and Communication': 'information_communication',
    'Financial and Insurance Activities': 'financial_insurance',
    'Real Estate Activities': 'real_estate',
    'Professional, Scientific and Technical Activities': 'professional_scientific_technical',
    'Administrative and Support Services Activities': 'administrative_support_services',
    'Education except Public Education': 'education_except_public',
    'Human Health and Social Work Activities except Public Health Activities': 'human_health_social_work',
    'Arts, Entertainment and Recreation': 'arts_entertainment_recreation',
    'Other Service Activities except Activities of Membership Organizations': 'other_service_activities'
}
employment_non_regular_by_industry['industry_category'] = employment_non_regular_by_industry['industry_category'].replace(industry_mapping)

# Rename columns for business size dataset
employment_non_regular_by_business_size.rename(columns={
    'Employment Size': 'business_size_category',
    **other_category
}, inplace=True)

# Rename Business Sizes
business_size_mapping = {
    'All Sizes': 'all_sizes',
    '20 - 99 workers': 'small_size',
    '100 - 199 workers': 'medium_size',
    '200 workers or more': 'large_size'
}
employment_non_regular_by_business_size['business_size_category'] = employment_non_regular_by_business_size['business_size_category'].replace(business_size_mapping)


# Rename gender category
gender_mapping = {
    'Both Sexes': 'both_sexes',
    'Female': 'female'
}

employment_non_regular_by_region['gender_category'] = employment_non_regular_by_region['gender_category'].replace(gender_mapping)
employment_non_regular_by_industry['gender_category'] = employment_non_regular_by_industry['gender_category'].replace(gender_mapping)
employment_non_regular_by_business_size['gender_category'] = employment_non_regular_by_business_size['gender_category'].replace(gender_mapping)

In [8]:
import numpy as np
import pandas as pd

worker_columns = [
    'total_non_regular', 'probationary', 'casual',
    'contractual', 'seasonal', 'apprentices_learners'
]

# Function to process and calculate male values
def calculate_male_rows(df, group_by_column):
    male_count = df[df['gender_category'] == 'male'].shape[0]
    both_sexes_count = df[df['gender_category'] == 'both_sexes'].shape[0]

    if male_count >= both_sexes_count:
        print("The code was already run.")
        return df

    male_rows = []

    for index, row in df.iterrows():
        if row['gender_category'] == 'both_sexes':
            female_row = df[
                (df[group_by_column] == row[group_by_column]) & 
                (df['year'] == row['year']) & 
                (df['gender_category'] == 'female')
            ]

            if not female_row.empty:
                male_row = row.copy()
                male_row['gender_category'] = 'male'

                for col in worker_columns:
                    female_value = female_row[col].values[0]

                    if female_value == 0:
                        male_value = 0
                    elif pd.isna(female_value):
                        male_value = row[col]
                    else:
                        male_value = row[col] - female_value

                    male_row[col] = male_value

                male_rows.append(male_row)

    male_df = pd.DataFrame(male_rows)
    return pd.concat([df, male_df], ignore_index=True)

employment_non_regular_by_industry = calculate_male_rows(employment_non_regular_by_industry, 'industry_category')
employment_non_regular_by_business_size = calculate_male_rows(employment_non_regular_by_business_size, 'business_size_category')


In [9]:
# Rearrange the csv files alphabetically by their firts column
employment_region_sorted = employment_non_regular_by_region.sort_values(by=['year', 'gender_category'], ascending=[True, True])
employment_industry_sorted = employment_non_regular_by_industry.sort_values(by=['year', 'gender_category'], ascending=[True, True])
business_size_sorted = employment_non_regular_by_business_size.sort_values(by=['year', 'gender_category'], ascending=[True, True])

# Save cleaned data to new CSV files
employment_region_sorted.to_csv('datasets/cleaned_region.csv', index=False)
employment_industry_sorted.to_csv('datasets/cleaned_industry.csv', index=False)
business_size_sorted.to_csv('datasets/cleaned_business_size.csv', index=False)
